### Hypothesis Testing Case Study

In [4]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [2]:
pwd()

'C:\\Users\\Rohan\\Documents\\JUPYTER_ROHAN\\JUPYTER\\Jupyter\\ROHAN_ALL\\SM_&_ML\\Data_preprocessing\\Anova_vs_T_test'

In [3]:
cust=pd.read_csv("cust_seg.csv")

In [4]:
cust.head(5)

,custid,sex,AqChannel,region,Marital_status,segment,pre_usage,Post_usage_1month,Latest_mon_usage,post_usage_2ndmonth
0,70,0,4,1,1,1,57,52,49.2,57.2
1,121,1,4,2,1,3,68,59,63.6,64.9
2,86,0,4,3,1,1,44,33,64.8,36.3
3,141,0,4,3,1,3,63,44,56.4,48.4
4,172,0,4,2,1,2,47,52,68.4,57.2


In [5]:
cust.columns

Index(['custid', 'sex', 'AqChannel', 'region', 'Marital_status', 'segment',
       'pre_usage', 'Post_usage_1month', 'Latest_mon_usage',
       'post_usage_2ndmonth'],
      dtype='object')

In [5]:
cust.Latest_mon_usage.mean()

63.17400000000001

In [6]:
cust.Latest_mon_usage.std()

11.242137352892753

### One Sample T-Test


- statistic = t_value 

In [8]:
stats.ttest_1samp(a= cust.Latest_mon_usage,               # Sample data
                 popmean= 50)  # Pop mean

Ttest_1sampResult(statistic=16.57233752433133, pvalue=2.4963719280931583e-39)

In [9]:
cust.Latest_mon_usage.mean()

63.17400000000001

### Two Sample T-Test (Paired)

In [7]:
print(cust.pre_usage.mean())
print(cust.Post_usage_1month.mean())
print(cust.post_usage_2ndmonth.mean())

52.23
52.775
58.05250000000003


### --------------------------------How two sample paired t-test works @ backend -------------------------------------------

In [8]:
cust[['pre_usage','Post_usage_1month']]
cust['differ'] = cust.pre_usage - cust.Post_usage_1month

data = cust[['pre_usage','Post_usage_1month','differ']]

print('mean =',data.differ.mean(),'std =',data.differ.std()) 

mean = -0.545 std = 8.886666371366704


In [12]:
data.shape

(200, 3)

In [13]:
stats.ttest_1samp(a= data.differ, popmean = 0) 

Ttest_1sampResult(statistic=-0.8673065458794775, pvalue=0.3868186820914985)

#### --------------------------------------------------------------------------------------------------------------------------------------------------------

#### Two sample Paired T-test 

- Here we are comparing means of 2 variables(if they are equal, ie they are from same population distribution)

- The same can also be understood as 2 var are 2 classes on which a dependent var (spending) is dependent and the same can be solved by anova

In [9]:
#stats.ttest_rel(a = before, b = after)
stats.ttest_rel(a=cust.pre_usage,
                b=cust.Post_usage_1month)    # Assume samples have equal variance?

Ttest_relResult(statistic=-0.8673065458794775, pvalue=0.3868186820914985)

In [34]:
#stats.ttest_rel(a = before, b = after)
stats.ttest_rel(a=cust.pre_usage,
                b=cust.post_usage_2ndmonth)    # Assume samples have equal variance?

Ttest_relResult(statistic=-8.866832246938742, pvalue=4.295733828012836e-16)

### Two sample T-Test (Independent)

In [12]:
Males_spend = cust.Post_usage_1month[cust.sex==0]
FeMales_spend = cust.Post_usage_1month[cust.sex==1]

In [13]:
print(FeMales_spend.head(3))
print(Males_spend.head(3))

1     59
92    62
93    44
Name: Post_usage_1month, dtype: int64
0    52
2    33
3    44
Name: Post_usage_1month, dtype: int64


In [15]:
FeMales_spend.mean(),Males_spend.mean()

(54.99082568807339, 50.120879120879124)

In [44]:
FeMales_spend.std(),Males_spend.std()

(8.13371516959346, 10.305160697259263)

In [35]:
stats.ttest_ind(a= Males_spend,
                b= FeMales_spend,
                equal_var=True)    # Assume samples have equal variance?

Ttest_indResult(statistic=-3.7340738531536797, pvalue=0.00024625461203549315)

#### Using Anova - Here anova can do what t_test (Independent) was doing.

In [17]:
stats.f_oneway(Males_spend, FeMales_spend)

F_onewayResult(statistic=13.94330754080599, pvalue=0.0002462546120354903)

### ANOVA


- 1 way Anova is same as independent T_test(ttest_ind) with equal_var assumption
- Later also proved that 1_way_Anova is same as Anova_lm 

- To test whether **customer segment(Indep. cat var)** affects the **Latest_month_usage(Dependent continuos var)**

In [30]:
cust.segment.value_counts()

2    105
3     50
1     45
Name: segment, dtype: int64

In [54]:
s1 = cust.Latest_mon_usage[cust.segment==1]
s2 = cust.Latest_mon_usage[cust.segment==2]
s3 = cust.Latest_mon_usage[cust.segment==3]

# Perform the ANOVA
stats.f_oneway(s1, s2, s3)

F_onewayResult(statistic=29.279283801321778, pvalue=7.36401083352674e-12)

In [52]:
cust.Latest_mon_usage.mean()
cust.Latest_mon_usage.std()

11.242137352892753

In [53]:
s1.mean(),s2.mean(),s3.mean()

(60.026666666666685, 68.08000000000003, 55.703999999999986)

#### Statsmodel.Api Anova vs scipy Anova
https://pythonfordatascience.org/anova-python/

In [1]:
import pandas as pd
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
    
import matplotlib.pyplot as plt

# Loading data
df = pd.read_csv("https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/difficile.csv")
df.drop('person', axis= 1, inplace= True)

# Recoding value from numeric to string
df['dose'].replace({1: 'placebo', 2: 'low', 3: 'high'}, inplace= True)
    
# Gettin summary statistics
rp.summary_cont(df['libido'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,libido,15.0,3.466667,1.76743,0.456349,2.487896,4.445437


In [65]:
df

,dose,libido
0,placebo,3
1,placebo,2
2,placebo,1
3,placebo,1
4,placebo,4
5,low,5
6,low,2
7,low,4
8,low,2
9,low,3


In [41]:
rp.summary_cont(df['libido'].groupby(df['dose']))

,N,Mean,SD,SE,95% Conf.,Interval
dose,,,,,,
high,5,5.0,1.581139,0.707107,3.450484,6.549516
low,5,3.2,1.303840,0.583095,1.922236,4.477764
placebo,5,2.2,1.303840,0.583095,0.922236,3.477764


In [42]:
stats.f_oneway(df['libido'][df['dose'] == 'high'], 
             df['libido'][df['dose'] == 'low'],
             df['libido'][df['dose'] == 'placebo'])

F_onewayResult(statistic=5.11864406779661, pvalue=0.024694289538222603)

#### ANOVA with statsmodels

In [45]:
results = ols('libido ~ C(dose)', data=df).fit()
results.summary()

C:\Users\Rohan\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 libido   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     5.119
Date:                Sat, 29 Jun 2019   Prob (F-statistic):             0.0247
Time:                        19:46:59   Log-Likelihood:                -24.683
No. Observations:                  15   AIC:                             55.37
Df Residuals:                      12   BIC:                             57.49
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              5.0000      0.627      7.972      0.000       3.634       6.366
C(dose)[T.low]        -1.8000      0.887     -2.029      0.065      -3.732       0.132
C(dose)[T.placebo]    -2.8000      0.887     -3.157      0.008      -4.732      -0.868
==============================================================================
Omnibus:                        2.517   Durbin-Watson:                   2.408
Prob(Omnibus):                  0.284   Jarque-Bera (JB):                1.108
Skew:                           0.195   Prob(JB):                        0.575
Kurtosis:                       1.727   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
aov_table = sm.stats.anova_lm(results, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(dose),20.133333,2.0,5.118644,0.024694
Residual,23.600000,12.0,NaN,NaN


### Chi-squre Test

- To test whether var **customer_segment** & **customer_region** are colinear

In [34]:
t=pd.crosstab(cust.segment, cust.region, margins = True)
t

region,1,2,3,All
segment,,,,
1,16,20,9,45
2,19,44,42,105
3,12,31,7,50
All,47,95,58,200


In [57]:
stats.chi2_contingency(observed= t)

(16.60444164948934,
 0.055282939487992365,
 9,
 array([[ 10.575,  21.375,  13.05 ,  45.   ],
        [ 24.675,  49.875,  30.45 , 105.   ],
        [ 11.75 ,  23.75 ,  14.5  ,  50.   ],
        [ 47.   ,  95.   ,  58.   , 200.   ]]))

### Correlation

In [37]:
print np.corrcoef(cust.Latest_mon_usage, cust.Post_usage_1month)

[[ 1.          0.61744926]
 [ 0.61744926  1.        ]]


In [38]:
print stats.stats.pearsonr(cust.Latest_mon_usage, cust.Post_usage_1month)

(0.61744926448549187, 2.0866647416871388e-22)


#### Types of Correlation

- np.corrcoef is pearson coefficient 
    > np.corrcoef = `sigma(xi - Xmean)` * `sigma(yi - Ymean)` / `N`*`std(x)` * `std(y)` 


- There's also a spearman coefficient 



In [21]:
li = [[1,2,3,4,5,6],[1,3,8,4,5,6]]

In [4]:
li[0]

[1, 2, 3, 4, 5, 6]

In [22]:
np.corrcoef(li[0],li[1])

array([[1.        , 0.59416044],
       [0.59416044, 1.        ]])

In [23]:
stats.pearsonr(li[0],li[1])

(0.5941604449831431, 0.21363656361310884)

In [24]:
x = li[0]
y = li[1]

In [25]:
np.mean(x),np.mean(y)

(3.5, 4.5)

In [26]:
(sum((x - np.mean(x))*(y - np.mean(y))))/(6*np.std(x)*np.std(y))

0.5941604449831431

### Simple regression

In [ ]:
sm.OLS?

In [ ]:
 statsmodels.tools.add_constant?

In [ ]:
import statsmodels.api as sm
Y = cust.Latest_mon_usage
X = sm.add_constant(cust.Post_usage_1month )
mod = sm.OLS(Y, X)
res = mod.fit()
print res.summary()

### Anova Looper function

In [15]:
def Anova_looper(df):
    num = df._get_numeric_data().columns
    cat = df.select_dtypes(['object','category']).columns

    li = []
    lt = []
    for x in num:
        for y in cat:
            lt.append([x,y])
            dat = ols('{} ~ {}'.format(x,y),
                    data=df).fit()

            aov_table = sm.stats.anova_lm(dat, typ=2)
            result = aov_table[['F','PR(>F)']].iloc[0].tolist()
            li.append(result)

    # ---------------------------------

    key =pd.DataFrame(lt,columns=['DV_Num','IV_Cat'])
    key.head(2)

    value =pd.DataFrame(li,columns=['F_value','P_value'])
    value.head(2)

    df = pd.merge(key,value,left_index=True,right_index=True)
    df['relation'] = np.where(df.P_value<0.1,'Corelated','Not_Corelated')
    return df

In [17]:
Anova_looper(tips)

,DV_Num,IV_Cat,F_value,P_value,relation
0,total_bill,sex,5.188346,0.023612,Corelated
1,total_bill,smoker,1.791412,0.182010,Not_Corelated
2,total_bill,day,2.767479,0.042454,Corelated
3,total_bill,time,8.396303,0.004105,Corelated
4,tip,sex,1.926155,0.166456,Not_Corelated
5,tip,smoker,0.008506,0.926593,Not_Corelated
6,tip,day,1.672355,0.173589,Not_Corelated
7,tip,time,3.633815,0.057802,Corelated
8,size,sex,1.811408,0.179599,Not_Corelated
9,size,smoker,4.369723,0.037627,Corelated
